# Embeddings  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PragmaticsLab/NLP-course-FinTech/blob/master/seminars/2/2_embeddings.ipynb)

## Word2Vec

Векторные модели, которые мы рассматривали до этого (tf-idf, BOW), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей для каждого слова.

 Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.


### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

![](https://www.researchgate.net/profile/Daniel_Braun6/publication/326588219/figure/fig1/AS:652185784295425@1532504616288/Continuous-Bag-of-words-CBOW-CB-and-Skip-gram-SG-training-model-illustrations.png)

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. 

Векторное представление основывается на *контекстной близости*: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии: А относится к B так же, как C относится к D) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятора семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл (текстовый)
* .bin.gz — бинарный файл

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. **для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она**.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

Для начала импортируем необходимые библиотеки:

In [0]:
import urllib.request # библиотека для скачивания данных
import gensim # библиотека для загрузки и использвоания моделй w2v
from gensim.models import word2vec # непосредственно методы w2v

import warnings
warnings.filterwarnings("ignore")

In [2]:
# скачиваем модель ruscorpora_mystem_cbow_300 с сайта rusvectores

urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7fbae8ed2668>)

Загружаем скачанную модель. Обратите внимание, что мы скачали бинарный файл (.bin.gz), поэтому у функции ```load_word2vec_format()``` параметр ```binary=True``` 


In [0]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz' 

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

Получим вектор для слова "день".

In [13]:
day_vec = model_ru['день_S']
day_vec[:10]

array([-0.02580778,  0.00970898,  0.01941961, -0.02332282,  0.02017624,
        0.07275085, -0.01444375,  0.03316632,  0.01242602,  0.02833412],
      dtype=float32)

In [14]:
len(day_vec)

300

[Частеречные тэги](https://yandex.ru/dev/mystem/doc/grammemes-values-docpage/#grammemes-values__parts) (например, _S, тег части речи слова) нужны, поскольку это специфика скачанной модели - она была натренирована на словах, размеченных по частям речи (и лемматизированных). 

**NB!** В названиях моделей на `rusvectores` указано, какой тегсет (набор обозначений тегов) они используют (mystem, upos и т.д.)

Посмотрим на ближайших соседей следующей группы слов:

In [0]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [16]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S :  0.7165195941925049
месяц_S :  0.631048858165741
вечер_S :  0.5828739404678345
утро_S :  0.5676207542419434
час_S :  0.5605547428131104
минута_S :  0.5297019481658936
гекатомбеон_S :  0.4897990822792053
денек_S :  0.48224714398384094
полчаса_S :  0.48217129707336426
ночь_S :  0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S :  0.6946247816085815
утро_S :  0.57301926612854
ноченька_S :  0.5582467317581177
рассвет_S :  0.5553582906723022
ночка_S :  0.5351512432098389
полдень_S :  0.5334426164627075
полночь_S :  0.478694349527359
день_S :  0.4780748784542084
сумерки_S :  0.4390218257904053
фундерфун_S :  0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681 

Находим косинусную близость пары слов с помощью метода ```similarity()```:

In [17]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895611


**Упражнение**  
Подберите два слова, похожесть которых выше 0.5

In [23]:
model_ru.similarity('половец_S', 'печенег_S')

0.55011994

**Упражнение**  
Найдите пример многозначного слова, для которого в топ-10 похожих на него слов входят слова связанные с разными значениями

In [38]:
model_ru.most_similar(positive=['ручка_S'], topn=10)

[('рукоятка_S', 0.5644145607948303),
 ('рука_S', 0.540177583694458),
 ('ручонка_S', 0.5033916234970093),
 ('рычажок_S', 0.49882832169532776),
 ('пальчик_S', 0.480662077665329),
 ('ножка_S', 0.4743875563144684),
 ('авторучка_S', 0.46884289383888245),
 ('локоток_S', 0.46842196583747864),
 ('плечико_S', 0.4646531939506531),
 ('рукоять_S', 0.4615430533885956)]

У загруженной модели много различных функций. Например, можно решать задачи на семантическую близость.

Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

Для решения примера в качестве параметров метода ```most_similar()``` необходимо передать:
* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

*Замечание:* не забываем взять самый близкий элемент, для этого необходимо указать ```[0][0]```.

In [41]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S']))

[('пельмень_S', 0.3977898955345154), ('плов_S', 0.39217692613601685), ('лаваш_S', 0.3905729055404663), ('ветчина_S', 0.38722336292266846), ('лососина_S', 0.3827736973762512), ('колбаса_S', 0.38114017248153687), ('мясо_S', 0.3810117542743683), ('толокно_S', 0.3776426315307617), ('винегрет_S', 0.3727692663669586), ('пирог_S', 0.3712701201438904)]


Метод ```doesnt_match()``` находит "лишнее слово" в группе слов:

In [42]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

'ананас_S'

**Упражнение**  
Придумайте и проверьте с помощью методов most_similar и doesnt_match какие-нибудь аналогии

In [47]:
model_ru.most_similar(positive=['водка_S', 'россия_S'], negative=['франция_S'])

[('самогон_S', 0.5609264373779297),
 ('вино_S', 0.5605303049087524),
 ('самогонка_S', 0.5500626564025879),
 ('пиво_S', 0.5379058718681335),
 ('коньяк_S', 0.533240556716919),
 ('горилка_S', 0.5187708139419556),
 ('портвейн_S', 0.5063881278038025),
 ('шампанское_S', 0.49136099219322205),
 ('боржоми_S', 0.47112810611724854),
 ('шампанский_A', 0.4676359295845032)]

**Упражнение**  
Приведите пример трех слов w1, w2, w3, таких, что w1 и w2 являются синонимами, w1 и w3 являются антонимами, но при этом, similarity(w1, w2) < similarity(w1, w3)

In [49]:
model_ru.similarity('хороший_A', 'приемлемый_A')

0.23596986

In [50]:
model_ru.similarity('хороший_A', 'плохой_A')

0.74635214

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [51]:
# скачиваем датасет
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2020-04-11 13:08:23--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67281491 (64M) [text/plain]
Saving to: ‘unlabeledTrainData.tsv.1’

unlabeledTrainData. 100%[===================>]  64.16M   162MB/s    in 0.4s    

2020-04-11 13:08:23 (162 MB/s) - ‘unlabeledTrainData.tsv.1’ saved [67281491/67281491]



Загрузим датасет в датафрейм и посмотрим на него, делаем это с помощью уже привычной библиотеки **pandas**:

In [52]:
import pandas as pd

# считываем данные в формате csv
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [53]:
# проверяем, что все корректно загрузилось
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [61]:
print(data.review.iloc[0])

"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."


Нам необходимо отчистить данные от лишнего: убрать ссылки, html-разметку и небуквенные символы. Затем нужно привести все к нижнему регистру и токенизировать. 

На выходе мы хотим получить массив из предложений, каждое из которых представляет собой массив слов.

Импортируем необходимые библиотеки и методы:

In [54]:
import nltk.data # библиотека Natural Language Toolkit
import re   # библиотека для регулярных выражений
from bs4 import BeautifulSoup # библиотека для парсинга xml
from nltk.corpus import stopwords # стоп-слова из NLTK
from nltk.tokenize import sent_tokenize, RegexpTokenizer  # токенизаторы из NLTK
nltk.download('punkt') # для правильной работы токенизатора

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Функции для очистки данных:

In [0]:
def review_to_wordlist(review, remove_stopwords=False):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
    review_text = BeautifulSoup(review, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    words = review_text.lower().split()
    if remove_stopwords:
      # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

Проходим по всему датасету и парсим написанной выше функцией текст в списки слов, удаляя при этом лишнее:

In [58]:
sentences = []  

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


Посмотрим, что получилось:

In [59]:
print(len(sentences))
print(sentences[0])

528987
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [0]:
with open('clean_text.txt', 'w') as f:
  for sent in sentences:
    f.write(' '.join(sent) + '\n')

In [100]:
!head clean_text.txt

watching time chasers it obvious that it was made by a bunch of friends
maybe they were sitting around one day in film school and said hey let s pool our money together and make a really bad movie or something like that
what ever they said they still ended up making a really bad movie dull story bad script lame acting poor cinematography bottom of the barrel stock music etc
all corners were cut except the one that would have prevented this film s release
life s like that
i saw this film about years ago and remember it as being particularly nasty
i believe it is based on a true incident a young man breaks into a nurses home and rapes tortures and kills various women it is in black and white but saves the colour for one shocking shot at the end the film seems to be trying to make some political statement but it just comes across as confused and obscene avoid
minor spoilersin new york joan barnard elvire audrey is informed that her husband the archeologist arthur barnard john saxon was my

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [62]:
print("Training model...")
# обучаем модель с векторами размерности 300, длиной окна 10
%time model_en = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 4min 25s, sys: 932 ms, total: 4min 26s
Wall time: 2min 23s


Смотрим, сколько в модели слов.

In [63]:
print(len(model_en.wv.vocab))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [66]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7826313972473145)]
[('men', 0.6039944291114807)]
[('canada', 0.7392696142196655), ('greece', 0.7387787103652954), ('europe', 0.7308732271194458)]
novel


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако, можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [67]:
model_en.wv.similarity('lion', 'rabbit')

0.3289393

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [68]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2020-04-11 13:22:06--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 163.70K  --.-KB/s    in 0.04s   

2020-04-11 13:22:06 (4.33 MB/s) - ‘alice.txt’ saved [167631/167631]



In [69]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# убираем переносы строк, токенизируем текст

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

# убираем всю пунктуацию и делим текст на слова по пробелу
for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель (сохраненные при обучении веса и параметры модели, то есть обект самой модеи), а `KeyedVectors` (просто пары "слово - вектор") — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [71]:
model_path = "movie_reviews.model"

# так можно сохранить модель для последующего дообучения
print("Saving model...")
model_en.save(model_path)

Saving model...


In [72]:
# загружаем нашу обученную модель и дообучаем на текстах "Алисы"

model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(97006, 150225)

Лев и кролик стали ближе друг к другу!

In [73]:
model.wv.similarity('lion', 'rabbit')

0.34829345

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [74]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка

Задача обучения модели w2v - это usupervised задача (обучение без учителя), "правильных" ответов нет, поэтому нельзя вычислить некую метрику качества, чтобы сравнить две модели между собой или просто по значению одной метрики сказать, насколько хороша полученная модель. 

Тем не менее, существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (пример про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия), получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [75]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

--2020-04-11 13:39:39--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871776 (851K) [text/plain]
Saving to: ‘ru_analogy_tagged.txt’

ru_analogy_tagged.t 100%[===================>] 851.34K  --.-KB/s    in 0.08s   

2020-04-11 13:39:39 (11.0 MB/s) - ‘ru_analogy_tagged.txt’ saved [871776/871776]



In [79]:
!head ru_analogy_tagged.txt

: capital-common-countries
афины_S греция_S багдад_S ирак_S
афины_S греция_S бангкок_S таиланд_S
афины_S греция_S пекин_S китай_S
афины_S греция_S берлин_S германия_S
афины_S греция_S берн_S швейцария_S
афины_S греция_S каир_S египет_S
афины_S греция_S канберра_S австралия_S
афины_S греция_S ханой_S вьетнам_S
афины_S греция_S гавана_S куба_S


Посчитаем точность модели на датасете с аналогиями:

In [0]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

In [84]:
len(res[1]['correct'])

52

In [86]:
len(res[1]['incorrect'])

463

In [0]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


## Визуализация

Еще один хороший способ глазами оценить качество модели -  визуализировать ее, например, на плоскости.
### t-SNE

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).


Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [87]:
# импортируем необходимые библиотеки
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []

# строим простой словарь частотности употреблений всех слов
fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)
    
#оставляем только 1000 самых частых слов
for w in fd.most_common(1000):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = model[top_words]


['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'they', 'at', 'by', 'who', 'an', 'from', 'so', 'like', 'there', 'or', 'her', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can']


In [0]:
top_words_vec = model[top_words]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


Применяем преобразование t-SNE для векторов выбранных 1000 слов:

In [88]:
len(top_words_vec[0])

300

In [89]:
%%time
# инициализируем модель
tsne = TSNE(n_components=2, random_state=0)
# обучаем и применяем
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 14.6 s, sys: 56.1 ms, total: 14.7 s
Wall time: 7.86 s


In [91]:
top_words_tsne[0]

array([13.030951 , -6.7126684], dtype=float32)

In [90]:
!pip install bokeh

In [92]:
# код для отрисовки визуализации t-SNE, погружаться не надо, 
# просто много настроек конфигурации :)

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD (singular value decomposition, сингулярное разлолжение матрицы), и потом применять t-SNE.

Понизим размерность наших векторов с исходных 300 до 50 с помощью SVD:


In [0]:
# загружаем SVD из библиотеки sklearn
from sklearn.decomposition import TruncatedSVD

# инициализируем SVD
svd_50 = TruncatedSVD(n_components=200)

# обучаем и применяем разложение
top_words_vec_50 = svd_50.fit_transform(top_words_vec)

# обучаем и применяем TSNE
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

Нарисуем полученный результат:

In [96]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. 

Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). 

Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [97]:
# чтобы усатновить fasstext, можно склонировать его с репозитория 

! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

Cloning into 'fastText'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 3768 (delta 35), reused 46 (delta 15), pack-reused 3684
Receiving objects: 100% (3768/3768), 8.20 MiB | 32.54 MiB/s, done.
Resolving deltas: 100% (2354/2354), done.
Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2871212 sha256=a451d70e349787f46903cef959e952831680e686d6fb5ba6665ca505ae014b0b
  Stored in directory: /tmp/pip-ephem-wheel-cache-lsw0c4bf/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


Обучить свою модель можно аналогично w2v:


In [0]:
import fasttext

ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

Посмотрим на вектор для слова "movie":

In [0]:
ft_model.get_word_vector("movie")

array([-6.73057232e-03,  2.08648682e-01, -1.88893840e-01, -4.85788099e-02,
        6.45734444e-02,  1.29233301e-02, -2.47105390e-01, -6.73017651e-03,
       -8.79168790e-03,  8.76431391e-02,  1.43808842e-01,  8.35265443e-02,
       -1.74307749e-02,  3.79030369e-02,  3.74617018e-02, -9.29040164e-02,
       -5.69233031e-04, -9.27814022e-02,  1.15031146e-01,  1.22447662e-01,
        5.78474961e-02, -3.64004560e-02,  9.89502743e-02, -7.79608786e-02,
        9.53299105e-02,  2.64451802e-02,  1.28836200e-01,  9.55208484e-03,
       -9.69734341e-02, -1.23743024e-02,  7.17910826e-02,  3.13416235e-02,
       -2.42757481e-02, -3.46199162e-02, -1.83102563e-02, -2.79114731e-02,
        9.18026268e-02, -3.58579867e-02, -7.19081089e-02, -1.66802078e-01,
        7.83748850e-02,  9.05971229e-02, -9.21034664e-02, -3.60868834e-02,
        9.05119032e-02, -5.10555767e-02,  7.91546479e-02,  1.12251854e-02,
       -2.90307347e-02, -5.20794950e-02,  1.09920003e-01, -4.71951626e-02,
        6.38564071e-03, -

Метод ```get_nearest_neighbors``` возвращает самые похожие слова (аналог метода ```most_similar()``` для w2v):

In [0]:
ft_model.get_nearest_neighbors('actor')

[(0.999921977519989, 'actors'),
 (0.9998977780342102, 'director'),
 (0.9998862743377686, 'directors'),
 (0.9998846650123596, 'perfectly'),
 (0.9998726844787598, 'predator'),
 (0.9998723268508911, 'utterly'),
 (0.9998608827590942, 'dialog'),
 (0.9998583793640137, 'actual'),
 (0.9998502135276794, 'attractive'),
 (0.9998490810394287, 'dialogue')]

С помощью метода ```get_analogues()``` можно получить аналогии:

In [0]:
ft_model.get_analogies("woman", "man", "actor")

[(0.9998149275779724, 'act'),
 (0.9998092651367188, 'badly'),
 (0.9998079538345337, 'actually'),
 (0.9997990131378174, 'poorly'),
 (0.999788761138916, 'fly'),
 (0.999783992767334, 'write'),
 (0.9997831583023071, 'absolutely'),
 (0.9997802972793579, 'simply'),
 (0.9997757077217102, 'silly'),
 (0.9997753500938416, 'enjoyable')]

Проблема с опечатками решена!

In [0]:
ft_model.get_nearest_neighbors('actr')

[(0.9999391436576843, 'act'),
 (0.9998903274536133, 'actors'),
 (0.9998863339424133, 'actor'),
 (0.9998792409896851, 'actress'),
 (0.9998623728752136, 'single'),
 (0.9998517632484436, 'actual'),
 (0.9998226761817932, 'terrible'),
 (0.9998196363449097, 'exact'),
 (0.9998190402984619, 'plot'),
 (0.9998172521591187, 'wrong')]

Проблема с out of vocabulary словами тоже решена!


In [0]:
ft_model.get_nearest_neighbors('moviegeek')

[(0.9999324679374695, 'reviews'),
 (0.9999246597290039, 'review'),
 (0.9999151825904846, 'recommended'),
 (0.9999132752418518, 'rented'),
 (0.9998916387557983, 'waste'),
 (0.999889075756073, 'movie'),
 (0.9998835921287537, 'thank'),
 (0.9998812079429626, 'not'),
 (0.9998751878738403, 'watchable'),
 (0.9998645782470703, 'only')]

# Применим полученные выше навыки и решим простую задачу анализа тональности твиттов:

Проделаем весь пайплайн от сырых текстов до получения обученной модели.
Отдельно скачиваем файлы с положительно окрашенными твитами и негативно окрашеннными. 
Это реальные данные русскоязычного сегмента твиттера.


In [0]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

--2019-09-08 11:18:01--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2019-09-08 11:18:02--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdc27882d59b7fbb4ee5d53213f.dl.dropboxusercontent.com/cd/0/inline/AoKSln6uG-7jh4tWkOpriQZeCuEWcU3dtZD2B0Tl6ZmHPmemBlq3X3Cd8QHqM03LlTapMG_xDeAdozZnk-4zhZzHCQRw9SoIDyNO-8oqtxXNQw/file# [following]
--2019-09-08 11:18:02--  https://ucdc27882d59b7fbb4ee5d53213f.dl.dropboxusercontent.com/cd/0/inline/AoKSln6uG-7jh4tWkOpriQZeCuEWcU3dtZD2B0Tl6ZmHPmemBlq3X3Cd8QHqM03LlTapMG_xDeAdozZnk-4zhZzHCQRw9SoIDyNO-8oqtxXNQw/file
Resolving ucdc

In [0]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

--2019-09-08 11:18:05--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2019-09-08 11:18:06--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb96cff5d4b82d5d36bc949e0a2.dl.dropboxusercontent.com/cd/0/inline/AoJVp2lAGycoFn43QrVjalbNX5WsQ2E_SEQsbfeb5NaBQgPxBB2wYp4G5GBL8v16jXq_YspfbwclseHSwwq6VmVp9GUnhUnHYJmmsS-CHwD8-g/file# [following]
--2019-09-08 11:18:06--  https://ucb96cff5d4b82d5d36bc949e0a2.dl.dropboxusercontent.com/cd/0/inline/AoJVp2lAGycoFn43QrVjalbNX5WsQ2E_SEQsbfeb5NaBQgPxBB2wYp4G5GBL8v16jXq_YspfbwclseHSwwq6VmVp9GUnhUnHYJmmsS-CHwD8-g/file
Resolving ucb9

In [0]:
import pandas as pd # библиотека для удобной работы с датафреймами
# загрузим и посмотрим на наш датасет

# загружаем положительные твитты
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive) # расставляем метки

# загружаем отрицательные твитты
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative) # расставляем метки

# соединяем два набора данных
df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [0]:
len(df)

226834

Проведем стандартный препроцессинг:

In [0]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 3.4MB/s 


In [0]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

# pymorphy2 - библиотека методов для морфологического анализа (в том числе лемматизации) русскоязычного текста
m = pymorphy2.MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [0]:
#@lru_cache(maxsize=128)
# если вы работаете не колабе, можно заменить pymorphy на mystem и раскомментирвать первую строку про lru_cache
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "    

In [0]:
def clean_text(text):
    return lemmatize(words_only(text))

In [0]:
# распараллелим процесс на 8 копий, чтобы ускорить, 
# и к каждому объекту датасета ( = твиту) применим написанную выше функцию препроцессинга

with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.head()

,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first_timee хоть я и школотый но поверь у мы т...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt katiacheh: ну ты идиотка) я испугаться за т...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,rt digger : кто то в угол сидеть и погибать от...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina_dyshkant вот что значит страшилка :d но ...


Запишем полученные данные в формате для обучения классификатора:

In [0]:
# переводим данные из датафрейма в списки

X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


Мы исользуем для классификации библиотеку fasstext, для этого ей нужно подать данные на вход в особенном формате: текстовый файл, в котором одна строка - один объект выборки, в формате 



```
__label__ 0 первый текст

__label__ 1 второй текст

__label__ 0 третий текст
```


и т.д.

Записываем train и test выборки в файлы в соответствии с форматом выше:

In [0]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

Обучаем классификатор fasttext:

In [0]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

In [1]:
dir(fasttext)

NameError: ignored

Смотрим на метрики качества (precision и recall) полученной модели:

In [0]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.8975499625948488
R@1: 0.8975499625948488
Number of examples: 74856


**Задание**

1. Обучите на корпусе твитов свою модель word2vec или возьмите готовую (предобученную) модель.
2. Обучите простую модель классификации (SVM, log-reg, etc) на полученных эмбеддингах (в качестве векторного представления всего твита возьмите средний эмбеддинг по всем словам).
2. (бонус) Вместо среднего эмбеддинга возьмите сумму, взвешенную весами tf-idf.
3. Сравните качество с результатами fastText. 